In [20]:
#Load some useful modules
import os
import numpy as np
import gzip, struct
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import pickle, gzip
import scipy.io as sio


# Load the raw files into a numpy array

In [153]:
personId="9"
eeg_s1 = sio.loadmat('./data/S'+personId+'_SR200_CE_EEG.ch.mat')
eeg_s1['EEG']['samp'][0][0].shape

(12000, 56)

In [154]:
mpl.get_backend()

'agg'

In [155]:
plt.clf()
plt.cla()
fig_size = plt.rcParams["figure.figsize"]
print "Current size:", fig_size
fig_size[0] = 10
fig_size[1] = 2
plt.rcParams["figure.figsize"] = fig_size
fig = plt.figure()
subEEG = eeg_s1['EEG']['samp'][0][0][:601]
plt.yticks([])
plt.imshow(subEEG.T)
plt.tight_layout()
plt.savefig("graph/eeg"+personId+".pdf",dpi=600)

Current size: [15.0, 25.0]


# Dataset Description

In [156]:
EEG=eeg_s1['EEG']['samp'][0][0]
EEG.shape

(12000, 56)

#Statistics

In [157]:
np.min(np.amin(EEG, axis=0).reshape(8,7)) 

-572.80000000000007

In [158]:
np.max(np.amax(EEG, axis=0).reshape(8,7))

544.0

In [159]:
np.round(np.mean(EEG, axis=0).reshape(8,7),2)

array([[-55.31, -20.68,  35.14, -38.07, -43.59,  15.9 ,  24.96],
       [ 55.38,  58.22, -21.19, -25.88,  63.32,  17.6 ,  32.2 ],
       [ -4.97,  -4.17, -26.98,  -5.94,  24.89,  20.75, -47.39],
       [  0.56,  57.88,  -1.76,  47.36, -29.63,  97.84,  23.56],
       [ -5.41, -43.08, -45.63, -78.34, -23.19,  66.35, -32.64],
       [ 10.31, -34.71, -16.46,   7.04,   8.09,  43.06,  24.87],
       [ -8.11, -15.53,  27.02, -13.6 ,  77.3 , -10.12,   5.84],
       [-40.43,  36.6 ,  30.97, -57.5 , -13.28,  59.71, -51.04]])

In [160]:
np.round(np.std(EEG, axis=0).reshape(8,7),2)

array([[ 245.43,   35.16,   72.82,   30.63,   67.21,   60.59,   41.29],
       [  37.04,   33.27,   44.  ,   66.44,   57.43,   53.7 ,   97.95],
       [  71.16,   22.23,   27.67,   31.32,   34.2 ,   39.69,   54.06],
       [  58.96,   25.3 ,   36.42,   28.67,   46.  ,   70.39,   50.28],
       [  33.26,  228.15,   49.49,  151.54,   28.53,   28.98,   29.72],
       [  30.96,   47.64,   42.31,   52.48,   34.91,   23.62,   31.8 ],
       [  27.19,  106.78,   31.35,   40.91,   38.85,   46.44,   34.63],
       [  38.95,   30.46,   27.68,   32.66,   33.3 ,   55.85,   27.73]])

In [161]:
plt.clf()
plt.cla()
fig_size[0] = 15
fig_size[1] = 25
plt.rcParams["figure.figsize"] = fig_size
fig = plt.figure()
for i in range(EEG.shape[1]):
    ax = fig.add_subplot(8, 7, i+1)
    plt.hist(EEG[:,i],50, range=[-355, 104],normed=1)
    ax.get_xaxis().set_ticks([-355, 0., 183])
    ax.get_yaxis().set_ticks([])
plt.show()
plt.tight_layout()
plt.savefig("graph/eeg_hist"+personId+".pdf",dpi=600)

In [162]:
EEG = EEG.astype(float)

In [163]:
EEG = (EEG - np.amin(EEG))/(np.amax(EEG) - np.amin(EEG))

In [164]:
#train_perm = np.random.permutation(EEG.shape[0])
#print(np.max(train_perm))
#EEG = EEG[train_perm,:]

In [165]:
EEG

array([[ 0.17147206,  0.53330946,  0.65087751, ...,  0.50743195,
         0.55802292,  0.49229943],
       [ 0.25993911,  0.53939828,  0.73818052, ...,  0.53169771,
         0.5784384 ,  0.50573066],
       [ 0.60467407,  0.56008238,  0.75170129, ...,  0.55023281,
         0.60145057,  0.53062321],
       ..., 
       [ 0.51110315,  0.46257163,  0.43776862, ...,  0.4803904 ,
         0.60010745,  0.45639327],
       [ 0.69108166,  0.47286891,  0.44645415, ...,  0.48809097,
         0.60673352,  0.46212393],
       [ 0.36604585,  0.45003582,  0.43114255, ...,  0.47394341,
         0.58864613,  0.4469914 ]])

In [166]:
# Write the data
newFile = open ("data/EEG"+personId+".dat", "wb")
newFile.write(struct.pack('i', EEG.shape[0]))
newFile.write(struct.pack('i', EEG.shape[1]))
EEG.astype('float').tofile(newFile)
newFile.close()